실습: 삼성전자 주가 데이터를 분석하는 어플리케이션을 function(tool)으로 수정하여, 다른 종목의 데이터를 정해진 내용대로(종목, 시작날짜, 종료날짜) 가져와서 답하는 어플리케이션을 작성하시오.


In [146]:
import os
import json
from pykrx import stock
from pykrx import bond
from openai import OpenAI

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [147]:
def get_stock_and_analyze(stockCode, startDate, endDate):
    print(f"\n Input StockNAme : {stockCode}\n Starting Date : {startDate}\n Ending Date : {endDate}")
    try:
        ticker = stock.get_market_ticker_name(stockCode)
        print(ticker)
    except Exception as e:
        return f"종목을 찾을수가 없습니다.: {e}"
    
    if not ticker:
        print("없는 코드입니다.")

    if not startDate :
        print("시작일자를 입력하세요.")

    if not endDate :
        print("끝나는일자를 입력하세요.")

    search_result = stock.get_market_ohlcv(startDate, endDate, stockCode)
    if search_result.empty :
        print("주가 Data를 찾을 수 없습니다.")
        
         
    stock_info = {
        "ticker": ticker,
        "startDate": startDate,
        "endDate": endDate,
        "min_price": int(search_result['저가'].min()),
        "min_price": int(search_result['고가'].max()),
    }
    return json.dumps(stock_info, ensure_ascii=False)



messages = [{"role": "user", "content": "삼성중공업 주식을 알려줘."}]
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_stock_and_analyze",
            "description": "특정 주식의 정보를 알려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "stockCode": {
                        "type": "string",
                        "description": "주식이름. 예) 005930",
                    },
                    "startDate": {
                        "type": "string", 
                        "description": "검색시작날짜. 예) 20240101"
                    },
                    "endDate": {
                        "type": "string", 
                        "description": "검색종료날짜. 예) 20241231"
                    },
                },
            "required": ["stockCode","startDate","endDate"],
        },
        }
        
    }
]

In [151]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

messages= [
  {
    "role": "user",
    "content": "(005930)의 2024년 12월부터 2025년 1월까지 주가를 분석해줘."
  }
]

response = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=messages,
  temperature=0,
  max_tokens=1024,
  tools=tools,
  tool_choice="auto" if tools is not None else None,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_153MEueAfcoZf6eR9gKvnAGM', function=Function(arguments='{"stockCode":"005930","startDate":"20241201","endDate":"20250131"}', name='get_stock_and_analyze'), type='function')])


In [154]:
if response_message.function_call is not None:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_stock_and_analyze": get_stock_and_analyze,
    }
    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        stockCode=function_args['stockCode'],
        startDate=function_args['startDate'],
        endDate=function_args['endDate'],
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
   


 Input StockNAme : 278470
 Starting Date : 20241201
 Ending Date : 20250131
에이피알


In [155]:
second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    
if not second_response:
    print("값이 없습니다.")

print(second_response.choices[0].message.content)

죄송하지만, (주)삼성중공업의 구체적인 2024년 12월부터 2025년 1월까지의 주가 분석 데이터는 현재 제공할 수 없습니다. 대신, 삼성중공업의 최신 소식이나 업계 동향에 대해 질문하시면 더 도움을 드릴 수 있습니다. 다른 정보를 원하시면 말씀해 주세요!
